# Advanced Python

1. Iterator
   1. List comprehension
   2. Generator
   3. Generator expression
2. Stack frame
   1. `frame` object
3. Customizing module import with `sys.meta_path`
4. Descriptor
   1. Keep data on the instance
5. Metaclass
5. Type introspection and abstract base class (abc)

In [ ]:
import sys
import os
import pprint

# Iterator

While processing data in memory, they are iterated one by one.  Assume we have 10 elements in a list.

In [ ]:
data = list(range(10))
print(data, type(data))

Python uses the [iterator protocol](https://docs.python.org/3/library/stdtypes.html#iterator-types) to get one element a time:

In [ ]:
class ListIterator:

    def __init__(self, data):
        self.data = data
        self.it = None

    def __iter__(self):
        return self

    def __next__(self):
        if None is self.it:
            self.it = 0
        elif self.it >= len(self.data)-1:
            raise StopIteration
        else:
            self.it += 1
        return self.data[self.it]

The `for ... in ...` construct applies to the iterator object.  Every time the construct needs the next element, `ListIterator.__next__()` is called:

In [ ]:
list_iterator = ListIterator(data)
print(list_iterator)
print(dir(list_iterator))
for i in list_iterator:
    print(i)

Of course, you don't really need to write your own `ListIterator` for iterating a list, because Python builds in an iterator already:

In [ ]:
list_iterator2 = iter(data)
print(list_iterator2)
print(dir(list_iterator2))
for i in list_iterator2:
    print(i)

The built-in iterator is created by calling the `__iter__()` method on the container object (`iter()` simply does it for you):

In [ ]:
list_iterator3 = data.__iter__()
print(list_iterator3)
for i in list_iterator3:
    print(i)

And the `for ... in ...` construct actually knows about the iterator protocol:

In [ ]:
for i in data:
    print(i)

## List comprehension

List comprehension is the construct `[... for ... in ...]`.  Python borrowed the syntax of list comprehension from other languages, e.g., Haskell, and it follows the iterator protocol.  It is very convenient.  For example, the above `for` loop can be replaced by a one-liner:

In [ ]:
print("\n".join([str(i) for i in data]))

## Generator

In [ ]:
def list_generator(input_data):
    for i in input_data:
        yield i

generator = list_generator(data)
print(generator)
print(dir(generator))
for i in list_generator(data):
    print(i)

## Generator expression

A more convenient way of creating a generator is to use the generator expression `(... for ... in ...)`.  Note this looks like the list comprehension `[... for ... in ...]`, but uses parentheses to replace the brackets.

In [ ]:
generator2 = (i for i in data)
print(generator2)
print(dir(generator2))
for i in generator2:
    print(i)

By using the generator expression, the data printing one-liner can drop the brackets:

In [ ]:
print("\n".join(str(i) for i in data))
# Compare the the list comprehension:
# print("\n".join( [ str(i) for i in data ] ))

# Python stack frame

(C)Python uses a stack-based interpreter.  We are allowed to peek all the previous stack frames:

In [ ]:
import traceback

traceback.print_stack()

## `frame` object

We can get the `frame` object of the current stack frame using [`inspect.currentframe()`](https://docs.python.org/3/library/inspect.html#inspect.currentframe):

In [ ]:
import inspect

f = inspect.currentframe()

A `frame` object has the following attributes:

* Namespace:
  * `f_builtins`: builtin namespace seen by this frame
  * `f_globals`: global namespace seen by this frame
  * `f_locals`: local namespace seen by this frame
* Other:
  * `f_back`: next outer frame object (this frame's caller)
  * `f_code`: code object being executed in this frame
  * `f_lasti`: index of last attempted instruction in bytecode
  * `f_lineno`: current line number in Python source code

In [ ]:
print([k for k in dir(f) if not k.startswith('__')])

We can learn many things about the frame in the object.  For example, we can take a look in the builtin namespace:

In [ ]:
print(f.f_builtins.keys())

A mysterious `code` object:

In [ ]:
print(f.f_code)

Because a `frame` object holds everything a construct uses, after finishing using the `frame` object, make sure to break the reference to it.  If we don't do it, it may take long time for the interpreter to break the reference for you.

In [ ]:
f.clear()
del f

In [ ]:
for it, fi in enumerate(inspect.stack()):
    sys.stdout.write('frame #{}:\n  {}\n\n'.format(it, fi))

```python
#!/usr/bin/env python3

import sys
import inspect

def main():
    for it, fi in enumerate(inspect.stack()):
        sys.stdout.write('frame #{}:\n  {}\n\n'.format(it, fi))

if __name__ == '__main__':
    main()
```

In [ ]:
!./showframe.py

# Customizing module import with `sys.meta_path`

Python [`importlib`](https://docs.python.org/3/library/importlib.html) allows high degree of freedom in customizing module import flow.  Here I will use an example to load a module, `onemod`, locating in an alternate directory, `altdir/`, and ask Python to load it from the non-standard location.

In [ ]:
# Bookkeeping code: keep the original meta_path.
old_meta_path = sys.meta_path[:]
#sys.meta_path = old_meta_path[:-1]

`importlib` provides many facilities.  The theme in this example is [`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path).  It defines a list of ['MetaPathFinder'](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder) objects for customizing the import process.

In [ ]:
sys.meta_path = old_meta_path
print(sys.meta_path)

At this point, the `onemod` cannot be imported, because `altdir/` is not in `sys.path`:

In [ ]:
try:
    import onemod
except ModuleNotFoundError as e:
    traceback.print_exc()

In a normal Python course, you will be asked to modify `sys.path` to include `altdir/` for correctly import `onemod`.  That is uninteresting, so we will use `MetaPathFinder`.  Here we subclass the abstract base class (ABC) and override the `find_spec()` method, to tell it to load the `onemod` module at the place we specify.

For our path finder to work, we need to properly set up a [`ModuleSpec`](https://docs.python.org/3/library/importlib.html#importlib.machinery.ModuleSpec), and create a [`SourceFileLoader`](https://docs.python.org/3/library/importlib.html#importlib.machinery.SourceFileLoader) object for it.

In [ ]:
import importlib.abc
import importlib.machinery

class MyMetaPathFinder(importlib.abc.MetaPathFinder):
    def find_spec(self, fullname, path, target=None):
        if fullname == 'onemod':
            print('DEBUG: fullname: {} , path: {} , target: {}'.format(fullname, path, target))
            fpath = os.path.abspath('altdir/onemod.py')
            loader = importlib.machinery.SourceFileLoader('onemod', fpath)
            return importlib.machinery.ModuleSpec(fullname, loader, origin=fpath)
        else:
            return None

sys.meta_path = old_meta_path + [MyMetaPathFinder()]
print(sys.meta_path)

It only deals with `onemod`.  To test it, ask it to load a module that does not exist:

In [ ]:
try:
    import one_non_existing_module
except ModuleNotFoundError as e:
    traceback.print_exc()

But after the meta path finder is inserted into meta path, `onemod` can be loaded:

In [ ]:
import onemod
print("show content in onemod module:", onemod.content)

See the module we loaded.  Compare it with a 'normal module'.

In [ ]:
import re
print('onemod:', onemod)
print('re:', re)

And the module objects have an important field `__spec__`, which is the `ModuleSpec` we created:

In [ ]:
print('onemod.__spec__:', onemod.__spec__)
print('re.__spec__:', re.__spec__)

In [ ]:
# Bookkeeping code.
sys.modules.pop('onemod', None)

# Descriptor

The [descriptor protocol](https://docs.python.org/3/howto/descriptor.html) allows us to route attribute access to anywhere outside the class.

In [ ]:
class ClsAccessor:
    """Routing access to all instance attributes to the descriptor object."""

    def __init__(self, name):
        self._name = name
        self._val = None

    def __get__(self, obj, objtype):
        print('On object {} , retrieve: {}'.format(obj, self._name))
        return self._val

    def __set__(self, obj, val):
        print('On object {} , update: {}'.format(obj, self._name))
        self._val = val

class MyClass:
    x = ClsAccessor('x')

o = MyClass()

See the message printed while getting the attribute 'x':

In [ ]:
print(o.x)

Setting the attribute also shows a message:

In [ ]:
o.x = 10
print(o.x)

Because the attribute value is kept in the descriptor, and the descriptor is kept in the 'class' object, attributes of all instances of `MyClass` share the same value.

In [ ]:
o2 = MyClass()
print(o2.x) # Not None!

## Keep data on the instance

Having all instances sharing the attribute value isn't always desirable.  Descriptor protocol allows to bind the values to the instance too.

In [ ]:
class InsAccessor:
    """Routing access to all instance attributes to alternate names on the instance."""

    def __init__(self, name):
        self._name = name

    def __get__(self, obj, objtype):
        print('On object {} , retrieve: {}'.format(obj, self._name))
        varname = '_acs' + self._name
        if not hasattr(obj, varname):
            setattr(obj, varname, None)
        return getattr(obj, varname)

    def __set__(self, obj, val):
        print('On object {} , update: {}'.format(obj, self._name))
        varname = '_acs' + self._name
        return setattr(obj, varname, val)

class MyClass2:
    x = InsAccessor('x')

mo = MyClass2()

Create an instance to test the descriptor.

In [ ]:
print(mo.x)

In [ ]:
mo.x = 10
print(mo.x)

In a new instance, the value uses the initial value:

In [ ]:
mo2 = MyClass2()
print(mo2.x)

# Metaclass

Python class is also an object.

In [ ]:
class ClassIsObject:
    pass

print(ClassIsObject)
print(ClassIsObject.__dict__)

[Metaclass](https://docs.python.org/3/reference/datamodel.html#metaclasses) allows programmers to customize class creation.

In [ ]:
class AutoAccessor:
    """Routing access to all instance attributes to alternate names on the instance."""

    def __init__(self):
        self.name = None

    def __get__(self, obj, objtype):
        print('On object {} , retrieve: {}'.format(obj, self.name))
        varname = '_acs' + self.name
        if not hasattr(obj, varname):
            setattr(obj, varname, None)
        return getattr(obj, varname)

    def __set__(self, obj, val):
        print('On object {} , update: {}'.format(obj, self.name))
        varname = '_acs' + self.name
        return setattr(obj, varname, val)

class AutoAccessorMeta(type):

    def __new__(cls, name, bases, namespace):

        print('DEBUG before names:', name)
        print('DEBUG before bases:', bases)
        print('DEBUG before namespace:', namespace)

        for k, v in namespace.items():
            if isinstance(v, AutoAccessor):
                v.name = k

        # Create the class object for MyAutoClass.
        newcls = super(AutoAccessorMeta, cls).__new__(cls, name, bases, namespace)

        print('DEBUG after names:', name)
        print('DEBUG after bases:', bases)
        print('DEBUG after namespace:', namespace)

        return newcls

We will use the descriptor to test the metaclass.  The new descriptor class `AutoAccessor` doesn't take the attribute name in the constructor.  Instead, `AutoAccessorMeta` assigns the correct attribute name.

In [ ]:
class MyAutoClass(metaclass=AutoAccessorMeta):
    x = AutoAccessor()  # Note: no name is given.

ao = MyAutoClass()

In [ ]:
print(ao.x)

In [ ]:
ao.x = 20
print(ao.x)